In [1]:
import meshio
import numpy as np
from multiprocessing import Pool
import time

from ugs import *

In [2]:
infile = "australia/data/TAS.vtk"
outfile = "costed_tas.vtk"
mesh = meshio.read(infile)

max_distance = 250000

In [3]:
# We need to define a way to calculate cost
@functools.lru_cache(maxsize=1048576)
def distance_with_elevation_scaled(mesh, current, _next):
    # The travel_cost can be any function, including just the distance.
    # Here, we exagerate the elevation difference, to make changing elevation more costly
    if current == _next:
        return 0
    
    z_scaling = 100.  # 100. is a random number to pick, but has quite a big impact on the resulting paths.
    
    new_current = np.append(mesh.points[current][:2], mesh.point_data['Z'][current] * z_scaling)
    new_next    = np.append(mesh.points[_next][:2],   mesh.point_data['Z'][_next]   * z_scaling)
    
    return int(np.linalg.norm(new_current - new_next))  # return as Int, just for niceness

In [4]:
# We need to define a way to calculate cost
@functools.lru_cache(maxsize=1048576)
def elevation_only(mesh, current, _next):
    # Only take into account elevation changes for costs
    if current == _next:
        return 0
    return int(abs(mesh.point_data['Z'][current] - mesh.point_data['Z'][_next]))

In [5]:
# Choose which cost function you want
travel_cost = elevation_only

## Prepare for parallel execution

We want to use a parallel map function to calculate the cost for all points. We can make this easier by 'baking in' the parameters of a function we know, and leave only the `starting point` as a variable.

In [6]:
from functools import partial

# We bake in the mesh and travel_cost function into the get_from_point function
get_from_point_in_mesh = partial(get_from_point, mesh = mesh, travel_cost_function = travel_cost, max_distance = max_distance)

In [7]:
# Now we can use the get_from_point function with only a point ID as a parameter, by going via the new get_from_point_in_mesh function
# Here we do a test, and see the output format: (point, total cost of all paths to that point)
print(get_from_point_in_mesh(2087))

(2087, 0)


## Prepare input data

We don't want to calculate the LEC of a point below sea-level, so here we find all the points above sea-level, so they can be used as starting points.

In [9]:
points_above_sealevel = np.nonzero(mesh.point_data['Z'] >= 0)[0][::-1]

In [10]:
print("Total starting points available: ", points_above_sealevel.shape[0])

Total starting points available:  1093


In [11]:
# Output data variables
all_costs = []
mesh.point_data['cost'] = np.zeros_like(mesh.point_data['Z'])

# For parallel execution, we chunk the data up into increments (inc), so we can monitor the progress
start = 0
inc = 200
stop = inc

# Run on 11 CPUs
p = Pool(6)

while start <  points_above_sealevel.shape[0]-1:
    start_time = time.time()
    # Here we use a parallel map function to send out the chunk across the CPUs in the Pool
    costs = p.map(get_from_point_in_mesh, points_above_sealevel[start:stop])
    print("From ", start, " to ", stop, " took ", time.time() - start_time, "seconds, starting with point", points_above_sealevel[start], ". Percent complete: ", 100*(float(stop)/points_above_sealevel.shape[0]))
    # Save the data
    all_costs.extend(costs)
    
    # Write out data progressively, so we can see progress in Paraview
    for i in all_costs:
        mesh.point_data['cost'][i[0]] = i[1]
    meshio.write(outfile, mesh)
    
    # move to the next chunk of data
    start += inc
    stop += inc
    if stop >= points_above_sealevel.shape[0]:
        stop = points_above_sealevel.shape[0]-1
    

From  0  to  1093  took  352.0361285209656 seconds, starting with point 1233 . Percent complete:  100.0


## Output

Show some of the results:

In [12]:
print(all_costs[:100])

[(1233, 388546), (1232, 396033), (1231, 366848), (1230, 275144), (1229, 351601), (1228, 403732), (1227, 370973), (1226, 309370), (1225, 314897), (1224, 362415), (1223, 338362), (1222, 385760), (1221, 366955), (1220, 361125), (1219, 378063), (1218, 376485), (1217, 299097), (1216, 386183), (1215, 323881), (1214, 293327), (1213, 424314), (1212, 318023), (1211, 279148), (1210, 320421), (1209, 409621), (1208, 371290), (1207, 390943), (1206, 337262), (1205, 363440), (1204, 316439), (1203, 328847), (1202, 350940), (1201, 277293), (1200, 413668), (1199, 347367), (1198, 281646), (1197, 259260), (1196, 391765), (1195, 208224), (1194, 348554), (1193, 285698), (1192, 318832), (1191, 294011), (1190, 355852), (1189, 316082), (1188, 464904), (1187, 353228), (1186, 393963), (1185, 384489), (1184, 375009), (1183, 242491), (1182, 311374), (1181, 246675), (1180, 431749), (1179, 318995), (1178, 355958), (1177, 412989), (1176, 209451), (1175, 426113), (1174, 365660), (1173, 386324), (1172, 349338), (1171, 

# LRU cache impact

## With an LRU cache maxisze of 1
`From  0  to  1093  took  179.5063943862915 seconds, starting with point 1233 . Percent complete:  100.0`
## With an LRU cache maxisze of 1000000
`From  0  to  1093  took  352.0361285209656 seconds, starting with point 1233 . Percent complete:  100.0`